# Azure AI Search - Basic Search Operations

This interactive notebook demonstrates fundamental search operations in Azure AI Search.

## Learning Objectives
- Perform simple text searches
- Use different query types (phrase, wildcard, boolean)
- Process and format search results
- Handle errors gracefully
- Understand search scoring and relevance

## Prerequisites
- Azure AI Search service configured
- Environment variables set (AZURE_SEARCH_SERVICE_ENDPOINT, AZURE_SEARCH_API_KEY, AZURE_SEARCH_INDEX_NAME)
- Sample data indexed in your search service

## Setup and Imports

First, let's import the necessary libraries and set up our connection to Azure AI Search.

In [ ]:
import os
import json
import logging
import re
from typing import List, Dict, Any
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

# Setup logging for better debugging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

print("✅ Imports completed successfully!")

## Initialize Search Client

Let's create our search client and test the connection.

In [ ]:
# Configuration - Update these with your actual values or set environment variables
SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT", "https://your-service.search.windows.net")
SEARCH_API_KEY = os.getenv("AZURE_SEARCH_API_KEY", "your-api-key")
SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME", "your-index-name")

try:
    # Initialize the search client
    search_client = SearchClient(
        endpoint=SEARCH_ENDPOINT,
        index_name=SEARCH_INDEX_NAME,
        credential=AzureKeyCredential(SEARCH_API_KEY)
    )
    
    # Test connection by performing a simple search
    test_results = search_client.search(search_text="*", top=1)
    test_count = len(list(test_results))
    
    print("✅ Successfully connected to Azure AI Search!")
    print(f"📋 Connected to index: {SEARCH_INDEX_NAME}")
    print(f"🔗 Service endpoint: {SEARCH_ENDPOINT}")
    
except Exception as e:
    print(f"❌ Error initializing search client: {str(e)}")
    print("Please ensure your environment variables are set correctly:")
    print("- AZURE_SEARCH_SERVICE_ENDPOINT")
    print("- AZURE_SEARCH_API_KEY")
    print("- AZURE_SEARCH_INDEX_NAME")

## 1. Simple Text Search

Let's start with the most basic type of search - a simple text query.

In [ ]:
def simple_search(query: str, top: int = 5):
    """Perform a simple text search and display results"""
    try:
        print(f"🔍 Searching for: '{query}'")
        print("-" * 50)
        
        # Perform the search
        results = search_client.search(
            search_text=query,
            top=top,
            include_total_count=True
        )
        
        # Process and display results
        result_count = 0
        for result in results:
            result_count += 1
            title = result.get('title', 'No title')
            score = result['@search.score']
            author = result.get('author', 'Unknown')
            
            print(f"{result_count}. {title}")
            print(f"   Score: {score:.3f}")
            print(f"   Author: {author}")
            
            # Show content preview if available
            content = result.get('content', '')
            if content:
                preview = content[:150] + '...' if len(content) > 150 else content
                print(f"   Preview: {preview}")
            
            print()
        
        if result_count == 0:
            print("No results found.")
        else:
            print(f"📊 Displayed {result_count} results")
            
    except Exception as e:
        print(f"❌ Search error: {str(e)}")

# Try some example searches
simple_search("python programming", top=3)

In [ ]:
# Try another search with different terms
simple_search("machine learning", top=3)

## 2. Phrase Search vs Individual Terms

Let's compare searching for an exact phrase versus individual terms.

In [ ]:
def compare_phrase_vs_terms(phrase: str, top: int = 3):
    """Compare exact phrase search vs individual terms"""
    print(f"🔤 Comparing search methods for: '{phrase}'")
    print("=" * 60)
    
    # Exact phrase search (with quotes)
    print("\n1️⃣ EXACT PHRASE SEARCH:")
    print(f"Query: \"{phrase}\"")
    print("-" * 30)
    
    try:
        phrase_results = search_client.search(
            search_text=f'"{phrase}"',
            top=top
        )
        
        phrase_count = 0
        for result in phrase_results:
            phrase_count += 1
            print(f"{phrase_count}. {result.get('title', 'No title')} (Score: {result['@search.score']:.3f})")
        
        if phrase_count == 0:
            print("No exact phrase matches found.")
            
    except Exception as e:
        print(f"❌ Phrase search error: {str(e)}")
    
    # Individual terms search
    print("\n2️⃣ INDIVIDUAL TERMS SEARCH:")
    print(f"Query: {phrase}")
    print("-" * 30)
    
    try:
        terms_results = search_client.search(
            search_text=phrase,
            top=top
        )
        
        terms_count = 0
        for result in terms_results:
            terms_count += 1
            print(f"{terms_count}. {result.get('title', 'No title')} (Score: {result['@search.score']:.3f})")
        
        if terms_count == 0:
            print("No results found for individual terms.")
            
    except Exception as e:
        print(f"❌ Terms search error: {str(e)}")
    
    print(f"\n📊 Summary: Phrase found {phrase_count} results, Terms found {terms_count} results")

# Compare different search approaches
compare_phrase_vs_terms("data science")

## 3. Boolean Search Operations

Azure AI Search supports boolean operators like AND, OR, and NOT.

In [ ]:
def demonstrate_boolean_search(term1: str, term2: str, top: int = 3):
    """Demonstrate boolean search operators"""
    print(f"🔗 Boolean Search Demo: '{term1}' and '{term2}'")
    print("=" * 60)
    
    boolean_queries = {
        "AND": f"{term1} AND {term2}",
        "OR": f"{term1} OR {term2}",
        "NOT": f"{term1} NOT {term2}"
    }
    
    for operator, query in boolean_queries.items():
        print(f"\n{operator} Operation:")
        print(f"Query: {query}")
        print("-" * 30)
        
        try:
            results = search_client.search(
                search_text=query,
                top=top
            )
            
            result_count = 0
            for result in results:
                result_count += 1
                title = result.get('title', 'No title')
                score = result['@search.score']
                print(f"{result_count}. {title} (Score: {score:.3f})")
            
            if result_count == 0:
                print("No results found.")
            else:
                print(f"Found {result_count} results")
                
        except Exception as e:
            print(f"❌ Error with {operator} search: {str(e)}")

# Demonstrate boolean operations
demonstrate_boolean_search("python", "tutorial")

## 4. Wildcard Search

Use wildcards (*) for partial matching.

In [ ]:
def demonstrate_wildcard_search(base_term: str, top: int = 3):
    """Demonstrate wildcard search patterns"""
    print(f"🃏 Wildcard Search Demo: '{base_term}'")
    print("=" * 50)
    
    wildcard_patterns = {
        "Prefix": f"{base_term}*",
        "Suffix": f"*{base_term}",
        "Contains": f"*{base_term}*"
    }
    
    for pattern_name, pattern in wildcard_patterns.items():
        print(f"\n{pattern_name} Pattern:")
        print(f"Query: {pattern}")
        print("-" * 25)
        
        try:
            results = search_client.search(
                search_text=pattern,
                top=top
            )
            
            result_count = 0
            for result in results:
                result_count += 1
                title = result.get('title', 'No title')
                score = result['@search.score']
                print(f"{result_count}. {title} (Score: {score:.3f})")
            
            if result_count == 0:
                print("No results found.")
            else:
                print(f"Found {result_count} results")
                
        except Exception as e:
            print(f"❌ Error with {pattern_name} wildcard: {str(e)}")

# Try wildcard searches
demonstrate_wildcard_search("program")

## 5. Field-Specific Search

Search within specific fields and control which fields are returned.

In [ ]:
def demonstrate_field_search(query: str, top: int = 3):
    """Demonstrate field-specific search operations"""
    print(f"🎯 Field-Specific Search Demo: '{query}'")
    print("=" * 60)
    
    # Search in all fields (default)
    print("\n1️⃣ SEARCH ALL FIELDS:")
    print("-" * 25)
    
    try:
        all_results = search_client.search(
            search_text=query,
            top=top
        )
        
        all_count = 0
        for result in all_results:
            all_count += 1
            print(f"{all_count}. {result.get('title', 'No title')} (Score: {result['@search.score']:.3f})")
        
        print(f"Results in all fields: {all_count}")
        
    except Exception as e:
        print(f"❌ Error searching all fields: {str(e)}")
    
    # Search in specific fields
    field_searches = {
        "Title only": ["title"],
        "Content only": ["content"],
        "Title + Description": ["title", "description"]
    }
    
    for search_name, fields in field_searches.items():
        print(f"\n2️⃣ {search_name.upper()}:")
        print(f"Fields: {', '.join(fields)}")
        print("-" * 25)
        
        try:
            field_results = search_client.search(
                search_text=query,
                search_fields=fields,
                top=top
            )
            
            field_count = 0
            for result in field_results:
                field_count += 1
                print(f"{field_count}. {result.get('title', 'No title')} (Score: {result['@search.score']:.3f})")
            
            if field_count == 0:
                print("No results found in specified fields.")
            else:
                print(f"Results in {search_name.lower()}: {field_count}")
                
        except Exception as e:
            print(f"❌ Error searching {search_name.lower()}: {str(e)}")

# Demonstrate field-specific searches
demonstrate_field_search("python")

## 6. Search with Highlighting

Highlight matching terms in search results.

In [ ]:
def search_with_highlights(query: str, highlight_fields: List[str], top: int = 3):
    """Demonstrate search with result highlighting"""
    print(f"✨ Search with Highlighting: '{query}'")
    print(f"Highlight fields: {', '.join(highlight_fields)}")
    print("=" * 60)
    
    try:
        results = search_client.search(
            search_text=query,
            highlight_fields=",".join(highlight_fields),
            highlight_pre_tag="<mark>",
            highlight_post_tag="</mark>",
            top=top
        )
        
        result_count = 0
        for result in results:
            result_count += 1
            title = result.get('title', 'No title')
            score = result['@search.score']
            
            print(f"\n{result_count}. {title}")
            print(f"   Score: {score:.3f}")
            
            # Show highlights if available
            highlights = result.get('@search.highlights', {})
            if highlights:
                print("   Highlights:")
                for field, snippets in highlights.items():
                    for snippet in snippets:
                        print(f"     {field}: {snippet}")
            else:
                print("   No highlights available")
        
        if result_count == 0:
            print("No results found.")
        else:
            print(f"\n📊 Found {result_count} results with highlights")
            
    except Exception as e:
        print(f"❌ Error in highlighted search: {str(e)}")

# Demonstrate highlighting
search_with_highlights("machine learning", ["title", "content"], top=2)

## 7. Pagination Example

Handle large result sets with pagination.

In [ ]:
def demonstrate_pagination(query: str, page_size: int = 3, num_pages: int = 3):
    """Demonstrate pagination through search results"""
    print(f"📄 Pagination Demo: '{query}'")
    print(f"Page size: {page_size}, Pages to show: {num_pages}")
    print("=" * 60)
    
    for page in range(num_pages):
        skip = page * page_size
        
        print(f"\n📖 Page {page + 1} (skip={skip}, top={page_size}):")
        print("-" * 40)
        
        try:
            results = search_client.search(
                search_text=query,
                top=page_size,
                skip=skip,
                include_total_count=True
            )
            
            page_count = 0
            for result in results:
                page_count += 1
                title = result.get('title', 'No title')
                score = result['@search.score']
                print(f"{page_count}. {title} (Score: {score:.3f})")
            
            if page_count == 0:
                print("No more results available.")
                break
            else:
                print(f"Showing {page_count} results on this page")
                
        except Exception as e:
            print(f"❌ Error on page {page + 1}: {str(e)}")
            break

# Demonstrate pagination
demonstrate_pagination("tutorial", page_size=2, num_pages=3)

## 8. Error Handling and Validation

Demonstrate proper error handling for search operations.

In [ ]:
def safe_search_with_validation(query: str, top: int = 5):
    """Demonstrate comprehensive error handling and input validation"""
    print(f"🛡️ Safe Search with Validation: '{query}'")
    print("=" * 50)
    
    # Input validation
    if not query or not query.strip():
        print("❌ Error: Search query cannot be empty")
        return
    
    if len(query.strip()) < 2:
        print("❌ Error: Search query must be at least 2 characters long")
        return
    
    if len(query) > 1000:
        print("❌ Error: Search query is too long (max 1000 characters)")
        return
    
    # Sanitize query (remove potentially problematic characters)
    sanitized_query = re.sub(r'[<>]', '', query.strip())
    
    print(f"✅ Query validation passed")
    print(f"Original query: '{query}'")
    print(f"Sanitized query: '{sanitized_query}'")
    print("-" * 30)
    
    try:
        # Perform search with error handling
        results = search_client.search(
            search_text=sanitized_query,
            top=top,
            include_total_count=True
        )
        
        result_count = 0
        for result in results:
            result_count += 1
            title = result.get('title', 'No title')
            score = result['@search.score']
            print(f"{result_count}. {title} (Score: {score:.3f})")
        
        if result_count == 0:
            print("No results found. Try:")
            print("- Using different keywords")
            print("- Checking spelling")
            print("- Using broader search terms")
        else:
            print(f"\n✅ Successfully found {result_count} results")
            
    except HttpResponseError as e:
        if e.status_code == 400:
            print(f"❌ Bad request - check your query syntax: {sanitized_query}")
        elif e.status_code == 403:
            print("❌ Access denied - check your API key and permissions")
        elif e.status_code == 404:
            print("❌ Index not found - verify your index name")
        elif e.status_code == 503:
            print("❌ Service unavailable - try again later")
        else:
            print(f"❌ HTTP error {e.status_code}: {e.message}")
    
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")
        print("Please check your connection and try again.")

# Test with valid query
safe_search_with_validation("python programming", top=3)

print("\n" + "="*60)

# Test with invalid queries
print("\n🧪 Testing error handling:")
safe_search_with_validation("")  # Empty query
safe_search_with_validation("a")  # Too short
safe_search_with_validation("valid query with <script>")  # Potentially problematic characters

## 9. Search Result Analysis

Analyze and understand search results and scoring.

In [ ]:
def analyze_search_results(query: str, top: int = 10):
    """Analyze search results and provide insights"""
    print(f"📊 Search Results Analysis: '{query}'")
    print("=" * 60)
    
    try:
        results = search_client.search(
            search_text=query,
            top=top,
            include_total_count=True
        )
        
        # Collect results for analysis
        result_list = list(results)
        
        if not result_list:
            print("No results to analyze.")
            return
        
        # Extract scores for analysis
        scores = [result['@search.score'] for result in result_list]
        
        # Basic statistics
        print(f"📈 Score Statistics:")
        print(f"   Total results: {len(result_list)}")
        print(f"   Score range: {min(scores):.3f} - {max(scores):.3f}")
        print(f"   Average score: {sum(scores) / len(scores):.3f}")
        
        # Show score distribution
        print(f"\n📊 Score Distribution:")
        high_scores = [s for s in scores if s >= 1.0]
        medium_scores = [s for s in scores if 0.5 <= s < 1.0]
        low_scores = [s for s in scores if s < 0.5]
        
        print(f"   High relevance (≥1.0): {len(high_scores)} results")
        print(f"   Medium relevance (0.5-1.0): {len(medium_scores)} results")
        print(f"   Low relevance (<0.5): {len(low_scores)} results")
        
        # Analyze authors if available
        authors = [result.get('author', 'Unknown') for result in result_list]
        author_counts = {}
        for author in authors:
            author_counts[author] = author_counts.get(author, 0) + 1
        
        print(f"\n👥 Author Distribution:")
        for author, count in sorted(author_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"   {author}: {count} results")
        
        # Show top 3 results with details
        print(f"\n🏆 Top 3 Results:")
        for i, result in enumerate(result_list[:3], 1):
            title = result.get('title', 'No title')
            score = result['@search.score']
            author = result.get('author', 'Unknown')
            
            print(f"\n{i}. {title}")
            print(f"   Score: {score:.3f}")
            print(f"   Author: {author}")
            
            # Show available fields
            fields = [key for key in result.keys() if not key.startswith('@')]
            print(f"   Available fields: {', '.join(fields[:5])}{'...' if len(fields) > 5 else ''}")
            
    except Exception as e:
        print(f"❌ Analysis error: {str(e)}")

# Analyze search results
analyze_search_results("web development", top=8)

## 10. Interactive Search Playground

Try your own searches with this interactive function.

In [ ]:
def interactive_search():
    """Interactive search playground"""
    print("🎮 Interactive Search Playground")
    print("=" * 50)
    print("Try different search queries and see the results!")
    print("Examples:")
    print('- Simple: "python tutorial"')
    print('- Phrase: "\"machine learning\""')
    print('- Boolean: "python AND tutorial"')
    print('- Wildcard: "program*"')
    print("\nModify the query below and run the cell:")
    
    # You can modify this query and re-run the cell
    your_query = "artificial intelligence"
    
    print(f"\n🔍 Your search: '{your_query}'")
    print("-" * 40)
    
    try:
        results = search_client.search(
            search_text=your_query,
            top=5,
            include_total_count=True
        )
        
        result_count = 0
        for result in results:
            result_count += 1
            title = result.get('title', 'No title')
            score = result['@search.score']
            author = result.get('author', 'Unknown')
            
            print(f"{result_count}. {title}")
            print(f"   Score: {score:.3f} | Author: {author}")
            
            # Show a brief content preview
            content = result.get('content', '')
            if content:
                preview = content[:100] + '...' if len(content) > 100 else content
                print(f"   Preview: {preview}")
            print()
        
        if result_count == 0:
            print("No results found. Try different search terms!")
        else:
            print(f"📊 Found {result_count} results")
            
    except Exception as e:
        print(f"❌ Search error: {str(e)}")
        print("Try a different query or check your connection.")

# Run the interactive search
interactive_search()

## Summary and Next Steps

🎉 **Congratulations!** You've completed the basic search operations tutorial.

### What You've Learned:
- ✅ Simple text search operations
- ✅ Phrase vs individual term searches
- ✅ Boolean search operators (AND, OR, NOT)
- ✅ Wildcard pattern matching
- ✅ Field-specific searches
- ✅ Result highlighting
- ✅ Pagination techniques
- ✅ Error handling and validation
- ✅ Search result analysis

### Next Steps:
1. **Practice**: Try the interactive search playground with your own queries
2. **Explore**: Check out the language-specific code samples:
   - [Python Examples](../python/) - Complete Python implementations
   - [C# Examples](../csharp/) - .NET implementations
   - [JavaScript Examples](../javascript/) - Node.js/Browser implementations
   - [REST API Examples](../rest/) - Direct HTTP calls
3. **Advanced Topics**: Move on to Module 3: Index Management
4. **Build**: Apply these concepts in your own applications

### Useful Resources:
- [Azure AI Search Documentation](https://docs.microsoft.com/en-us/azure/search/)
- [Query Syntax Reference](https://docs.microsoft.com/en-us/azure/search/query-simple-syntax)
- [Python SDK Documentation](https://docs.microsoft.com/en-us/python/api/azure-search-documents/)

Happy searching! 🔍✨